<a href="https://colab.research.google.com/github/EmilyHong77/gentrification_in_montreal/blob/main/notebooks/data_standardization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Information**<br>
Crosswalk Source: <br>
https://github.com/jamaps/CLTD/tree/master/crosswalk_tables

This notebook performs standardization of census data across multiple years (2001 to 2021). Key steps include applying weighting methodology to population-based and dwelling-based variables (e.g., total and age-specific population counts, citizenship and migration status, marital status, education attainment, employment and unemployment, commuting modes, and language use), as well as household and dwelling characteristics (e.g., dwelling type, condition, and tenure).
Income and housing cost measures (e.g., median household income, average gross rent, and average dwelling value) are standardized through proportional weighting, while select indicators (e.g., population change %) require manual calculation. Placeholders are removed where census tracts do not have valid matches across years.
Steps vary for census years, depending on variable availability, weighting structure, or tract boundary changes across years.
This process prepares the datasets for Ding measurement application.

**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Library**

In [ ]:
import pandas as pd
import numpy as np

# 1996 Standardization

**1996→2021 Crosswalk Summary**: <br>

unique source_ctuid: 4219 <br>
unique target_ctuid: 5959 <br>
Final 1996 Standardized Table Shape: 5959 rows x 2 columns

In [ ]:
# Read in the data
census_1996 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/1996_clean.csv')

# Read in crosswalk table
cw_1996_2021 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/3_census_crosswalks/ct_1996_2021.csv')

In [ ]:
# Define CTUID column name
ctuidcol = "GeoUID_1996"

# w_pop variable
field_pop = [
    "Population_1996"
    ]

In [ ]:
# Remove placeholder CTUIDs (-1.0) before merging
cw_1996_2021 = cw_1996_2021[
    (cw_1996_2021['source_ctuid'] != -1.0) &
    (cw_1996_2021['target_ctuid'] != -1.0)
]

In [ ]:
# Joining the two input tables (following model code structure exactly)
merged = cw_1996_2021.merge(
    census_1996,
    how="outer",
    left_on="source_ctuid",
    right_on=ctuidcol,
    indicator=True
)

print(merged['_merge'].value_counts())

In [ ]:
# Apply population weight to single variable
single_field = field_pop[0]
weighted_col = f"w_{single_field}"
merged[weighted_col] = merged["w_pop"] * merged[single_field]

In [ ]:
# Group by the target census tracts, summing by the wanted fields
output_fields = [weighted_col]
output_data = merged.groupby(["target_ctuid"])[output_fields].agg("sum", min_count=1).reset_index()
output_data = output_data.rename(columns={"target_ctuid": "GeoUID_2021"})

# Drop w_ prefix
output_data.columns = [c[2:] if c.startswith("w_") else c for c in output_data.columns]

In [ ]:
print(output_data.shape)
print(output_data.head())

In [ ]:
# Export file to drive
output_data.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/4_standardized_data/1996_standardized.csv', index=False)

# 2001 Standardization

**2001→2021 Crosswalk Summary**:

unique source_ctuid: 4798<br>
unique target_ctuid: 6078<br>
Final 2001 Standardized Table Shape: 6078 rows x 67 columns

In [ ]:
# Read in the data
census_2001 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2001_clean.csv')

# Read in crosswalk table
cw_2001_2021 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/3_census_crosswalks/ct_2001_2021.csv')

**Weighting Methodology**

- **absolute values** (total counts) <br>
Apply weights using: <br>
• w_pop → for population-based variables (48 variables) <br>
• w_dwe → for dwelling-based variables (14 variables)
- **averages/medians** <br>
Use a weighted average, where each tract’s value is weighted by its population or number of dwellings. Since these are averages rather than totals, we apply proportional weighting instead of direct multiplication.<br>
• Median household income <br>
• Average gross rent <br>
• Average dwelling value
- **special handling**: <br>
Variable requiring manual calculation.  <br>
• Population change %

In [ ]:
# Define CTUID column name
ctuidcol = "GeoUID_2001"

# Categorize variables on calculation type (66 variables total excluding GeoUID and Area (sq km))
fields_pop = [
    "Population_2001",
    "Population 0 to 9_2001", "Population 10 to 19_2001", "Population 20 to 29_2001",
    "Population 30 to 39_2001", "Population 40 to 49_2001", "Population 50 to 59_2001",
    "Population 60 plus_2001", "Total male population_2001", "Total female population_2001",
    "Non-movers_2001", "Movers_2001", "Non-migrants_2001",
    "Migrants_2001", "Internal migrants_2001", "External migrants_2001", "Non-visible minority_2001",
    "Visible minority_2001", "Non-Canadian citizens_2001", "Canadian citizens_2001",
    "Non-immigrants_2001", "Immigrants_2001", "Non-permanent residents_2001", "Never married_2001",
    "Divorced_2001", "Widowed_2001", "Common-law couples_2001", "Married couples_2001",
    "Employed_2001", "Unemployed_2001",
    "No certificate or diploma_2001", "High school or Secondary degree_2001",
    "College or CEGEP degree_2001", "Trades certificate, diploma or apprenticeship_2001",
    "Bachelors degree or higher_2001",
    "Public transit_2001", "Walked_2001", "Bicycle_2001", "Transportation other methods_2001",
    "Transportation vehicle driver_2001", "Transportation vehicle non-driver_2001",
    "English only_2001", "French only_2001", "English and French_2001",
    "English and non-official language(s)_2001", "French and non-official language(s)_2001",
    "English, French, and non-official language(s)_2001", "Allophone_2001"
    ]

fields_dwe = [
    "Households_2001","Dwellings_2001", "Apartment with fewer than five stories_2001",
    "Apartment with five or more storeys_2001", "Row house_2001", "Single-detached house_2001",
    "Semi-detached house_2001", "Movable dwelling_2001", "Other single-attached house_2001",
    "Major repairs_2001","Regular maintenance_2001",
    "Owned housing_2001", "Rented housing_2001", "Band housing_2001",
    "Owner_2001", "Tenant_2001"
    ]

fields_weighted_avg = [
    "Median household income ($)_2001",
    "Average gross rent ($)_2001",
    "Average value dwelling ($)_2001"
    ]

fields_manual = [
    "Population change (%)_2001"
    ]

In [ ]:
# Combine all fields to convert to numeric
all_fields = fields_pop + fields_dwe + fields_weighted_avg + fields_manual

# Convert all columns except GeoUID to numeric
for col in all_fields:
  if col != ctuidcol:
    census_2001[col] = pd.to_numeric(census_2001[col], errors="coerce")

**Placeholders** (-1) are used in the crosswalk file to indicate census tracts that do not have a valid match across years. <br>
- For source_ctuid = -1, the weights (w_pop, w_dwe) are -1.
- For target_ctuid = -1, the weights are 0 (1.00E-10). <br>

Because these rows represent invalid or negligible mappings, we dropped all -1 placeholders before merging.

In [ ]:
# Remove placeholder CTUIDs (-1.0) before merging
cw_2001_2021 = cw_2001_2021[
    (cw_2001_2021['source_ctuid'] != -1.0) &
    (cw_2001_2021['target_ctuid'] != -1.0)
]

In [ ]:
# Joining the two input tables (following model code structure exactly)
merged = cw_2001_2021.merge(
    census_2001,
    how="outer",
    left_on="source_ctuid",
    right_on=ctuidcol,
    indicator=True
)

print(merged['_merge'].value_counts())


**Definitions** <br>
**splits**: a single source_ctuid (2001) mapping to multiple target_ctuid (2021) <br>
**merge**: a single target_ctuid (2021) receiving multiple source_ctuid (2001) <br>
<br>
**Verified Findings**<br>
2001 census tracts are a subset of the crosswalk tracts <br>
- **matches**: 862 GEOUID_2001 matched with 985 source_ctuid tracts; 3,936 additional unique source_ctuid in the crosswalk are unmatched. <br>
- **2001 splits**: 94 matched 2001 tracts split (69 split into 2 targets, 21 into 3, 4 into 4); 768 did not split. <br>
- **2021 merge**: No partial merges. Every merge target is either fully covered by matched sources or fully outside the subset (0 targets mixing matched + unmatched sources).

**w_pop and w_dwe**

In [ ]:
# Apply weights
output_fields = []

for f in fields_pop:
    weighted_col = f"w_{f}"
    merged[weighted_col] = merged["w_pop"] * merged[f]
    output_fields.append(weighted_col)

for f in fields_dwe:
    weighted_col = f"w_{f}"
    merged[weighted_col] = merged["w_dwe"] * merged[f]
    output_fields.append(weighted_col)

# Print numbers of rows and columns
print(merged.shape)

# Print column names
for col in merged.columns:
    print(col)

In [ ]:
# Group and aggregate weighted variables
weighted_data = merged.groupby("target_ctuid")[output_fields].agg("sum", min_count=1)

# Drop w_ prefix
weighted_data.columns = [c[2:] if c.startswith("w_") else c for c in weighted_data.columns]

# Print numbers of rows and columns
print(weighted_data.shape)

# Print column names
for col in weighted_data.columns:
    print(col)

**Next...**
- Calculate weighted average
- Manual calculation for special cases (obtain 1996 population)
- Inspect values and export dataset

**weighted average**

In [ ]:
# Initialize an empty DataFrame for weighted averages
weighted_avg_output = pd.DataFrame()

# For median household income (w_pop)
income_col = "Median household income ($)_2001"
weighted_income = (
    (merged[income_col] * merged["w_pop"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_pop"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[income_col] = weighted_income

# For average gross rent (w_dwe)
rent_col = "Average gross rent ($)_2001"
weighted_rent = (
    (merged[rent_col] * merged["w_dwe"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_dwe"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[rent_col] = weighted_rent

# For average dwelling value (w_dwe)
value_col = "Average value dwelling ($)_2001"
weighted_value = (
    (merged[value_col] * merged["w_dwe"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_dwe"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[value_col] = weighted_value

# Print column names
for col in weighted_avg_output.columns:
    print(col)

**Findings Update:** <br>
- **2021 merges**: 15 targets (within subset) are merges
  - full coverage: 14 merge targets have complete data for all average fields
  - partial coverage: 1 merge target has some sources missing average fields (averages computed from available sources)<br>
  target_ctuid: 4620314.0  
  present: 4620314.0 <br>
  missing: 4620315.0
  - No-data merges: 0 merge targets lack data for all average fields
- **avg-field (source lvl)**: 16 matched 2001 source_ctuid have no values for the average-type columns.<br>
These sources contribute nothing to average numerators/denominators; any 2021 target built only from such sources will have those averages NaN.

In [ ]:
# Combine weighted sums and weighed averages
final_2001_to_2021 = (
    weighted_data
    .join(weighted_avg_output, how="outer")
    .reset_index()
    .rename(columns={"target_ctuid": "GeoUID_2021"})
)

print(final_2001_to_2021.shape)
print(final_2001_to_2021.head())

**population change calculation**

In [ ]:
# Read in the standardized 1996 data
df1996 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/4_standardized_data/1996_standardized.csv')

# Merge 1996 population into the 2001-standardized table
final_2001_to_2021 = final_2001_to_2021.merge(
    df1996,
    on="GeoUID_2021",
    how="left"
)

# Calculate % change (1996 → 2001)
den = pd.to_numeric(final_2001_to_2021["Population_1996"], errors="coerce").replace(0, np.nan)
num = pd.to_numeric(final_2001_to_2021["Population_2001"], errors="coerce")
final_2001_to_2021["Population change (%)_2001"] = ((num - den) / den * 100).round(1)

# Quick check
print(final_2001_to_2021[["GeoUID_2021", "Population_1996", "Population_2001", "Population change (%)_2001"]].head())

In [ ]:
# Print column names
for col in final_2001_to_2021.columns:
    print(col)

In [ ]:
final = final_2001_to_2021.copy()

# Drop useless 1996 columns
final = final.drop(columns=["Population_1996_x", "Population_1996_y", "Population_1996"], errors="ignore")

print("Finalized table:", final.shape)
print(final.head(3))

In [ ]:
# Export file
final.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/4_standardized_data/2001_standardized.csv', index=False)

# 2006 Standardization

**2006→2021 Crosswalk Summary:** <br>

  unique source_ctuid: 5076 <br>
  unique target_ctuid: 6170 <br>
Final 2006 Standardized Table Shape: 6170 rows x 67 columns

In [ ]:
# Read in the data
census_2006 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2006_clean.csv')

# Read in crosswalk table
cw_2006_2021 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/3_census_crosswalks/ct_2006_2021.csv')

In [ ]:
# Define CTUID column name
ctuidcol = "GeoUID_2006"

# Categorize variables on calculation type (66 variables total excluding GeoUID and Area (sq km))
fields_pop = [
    "Population_2006",
    "Population 0 to 9_2006", "Population 10 to 19_2006", "Population 20 to 29_2006",
    "Population 30 to 39_2006", "Population 40 to 49_2006", "Population 50 to 59_2006",
    "Population 60 plus_2006", "Total male population_2006", "Total female population_2006",
    "Non-movers_2006", "Movers_2006", "Non-migrants_2006",
    "Migrants_2006", "Internal migrants_2006", "External migrants_2006", "Non-visible minority_2006",
    "Visible minority_2006", "Non-Canadian citizens_2006", "Canadian citizens_2006",
    "Non-immigrants_2006", "Immigrants_2006", "Non-permanent residents_2006", "Never married_2006",
    "Divorced_2006", "Widowed_2006", "Common-law couples_2006", "Married couples_2006",
    "Employed_2006", "Unemployed_2006",
    "No certificate or diploma_2006", "High school or Secondary degree_2006",
    "College or CEGEP degree_2006", "Trades certificate, diploma or apprenticeship_2006",
    "Bachelors degree or higher_2006",
    "Public transit_2006", "Walked_2006", "Bicycle_2006", "Transportation other methods_2006",
    "Transportation vehicle driver_2006", "Transportation vehicle non-driver_2006",
    "English only_2006", "French only_2006", "English and French_2006",
    "English and non-official language(s)_2006", "French and non-official language(s)_2006",
    "English, French, and non-official language(s)_2006", "Allophone_2006"
    ]

fields_dwe = [
    "Households_2006","Dwellings_2006", "Apartment with fewer than five stories_2006",
    "Apartment with five or more storeys_2006", "Row house_2006", "Single-detached house_2006",
    "Semi-detached house_2006", "Movable dwelling_2006", "Other single-attached house_2006",
    "Major repairs_2006","Regular maintenance_2006",
    "Owned housing_2006", "Rented housing_2006", "Band housing_2006",
    "Owner_2006", "Tenant_2006"
    ]

fields_weighted_avg = [
    "Median household income ($)_2006",
    "Average gross rent ($)_2006",
    "Average value dwelling ($)_2006"
    ]

fields_manual = [
    "Population change (%)_2006"
    ]

In [ ]:
# Combine all fields to convert to numeric
all_fields = fields_pop + fields_dwe + fields_weighted_avg + fields_manual

# Convert all columns except GeoUID to numeric
for col in all_fields:
  if col != ctuidcol:
    census_2006[col] = pd.to_numeric(census_2006[col], errors="coerce")

In [ ]:
# Remove placeholder CTUIDs (-1.0) before merging
cw_2006_2021 = cw_2006_2021[
    (cw_2006_2021['source_ctuid'] != -1.0) &
    (cw_2006_2021['target_ctuid'] != -1.0)
]

In [ ]:
# Joining the two input tables (following model code structure exactly)
merged = cw_2006_2021.merge(
    census_2006,
    how="outer",
    left_on="source_ctuid",
    right_on=ctuidcol,
    indicator=True
)

print(merged['_merge'].value_counts())

**w_pop and w_dwe**

In [ ]:
# Apply weights
output_fields = []

for f in fields_pop:
    weighted_col = f"w_{f}"
    merged[weighted_col] = merged["w_pop"] * merged[f]
    output_fields.append(weighted_col)

for f in fields_dwe:
    weighted_col = f"w_{f}"
    merged[weighted_col] = merged["w_dwe"] * merged[f]
    output_fields.append(weighted_col)

# Print numbers of rows and columns
print(merged.shape)

# Print column names
for col in merged.columns:
    print(col)

In [ ]:
# Group and aggregate weighted variables
weighted_data = merged.groupby("target_ctuid")[output_fields].agg("sum", min_count=1)

# Drop w_ prefix
weighted_data.columns = [c[2:] if c.startswith("w_") else c for c in weighted_data.columns]

# Print numbers of rows and columns
print(weighted_data.shape)

# Print column names
for col in weighted_data.columns:
    print(col)

**weighted average**

In [ ]:
# Initialize an empty DataFrame for weighted averages
weighted_avg_output = pd.DataFrame()

# For median household income (w_pop)
income_col = "Median household income ($)_2006"
weighted_income = (
    (merged[income_col] * merged["w_pop"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_pop"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[income_col] = weighted_income

# For average gross rent (w_dwe)
rent_col = "Average gross rent ($)_2006"
weighted_rent = (
    (merged[rent_col] * merged["w_dwe"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_dwe"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[rent_col] = weighted_rent

# For average dwelling value (w_dwe)
value_col = "Average value dwelling ($)_2006"
weighted_value = (
    (merged[value_col] * merged["w_dwe"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_dwe"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[value_col] = weighted_value

# Print column names
for col in weighted_avg_output.columns:
    print(col)

In [ ]:
# Combine weighted sums and weighed averages
final_2006_to_2021 = (
    weighted_data
    .join(weighted_avg_output, how="outer")
    .reset_index()
    .rename(columns={"target_ctuid": "GeoUID_2021"})
)

print(final_2006_to_2021.shape)
print(final_2006_to_2021.head())

**population change calculation**

In [ ]:
# Read in the standardized 2001 data
df2001 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/4_standardized_data/2001_standardized.csv')

# Merge 2001 population into the 2006-standardized table
final_2006_to_2021 = final_2006_to_2021.merge(
    df2001[['GeoUID_2021', 'Population_2001']],
    on="GeoUID_2021",
    how="left"
)

# Calculate % change (2001 → 2006)
den = pd.to_numeric(final_2006_to_2021["Population_2001"], errors="coerce").replace(0, np.nan)
num = pd.to_numeric(final_2006_to_2021["Population_2006"], errors="coerce")
final_2006_to_2021["Population change (%)_2006"] = ((num - den) / den * 100).round(1)

# Quick check
print(final_2006_to_2021[["GeoUID_2021", "Population_2001", "Population_2006", "Population change (%)_2006"]].head())

In [ ]:
final = final_2006_to_2021.copy()

# Drop useless 2001 columns
final = final.drop(columns=["Population_2001"], errors="ignore")

print("Finalized table:", final.shape)

for col in final.columns:
    print(col)

In [ ]:
# Export file
final.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/4_standardized_data/2006_standardized.csv', index=False)

# 2011 Standardization

In [ ]:
# Read in the data
census_2011 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2011_clean.csv')

# Read in crosswalk table
cw_2011_2021 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/3_census_crosswalks/ct_2011_2021.csv')

In [ ]:
# Define CTUID column name
ctuidcol = "GeoUID_2011"

# Categorize variables on calculation type (66 variables total excluding GeoUID and Area (sq km))
fields_pop = [
    "Population_2011",
    "Population 0 to 9_2011", "Population 10 to 19_2011", "Population 20 to 29_2011",
    "Population 30 to 39_2011", "Population 40 to 49_2011", "Population 50 to 59_2011",
    "Population 60 plus_2011", "Total male population_2011", "Total female population_2011",
    "Non-movers_2011", "Movers_2011", "Non-migrants_2011",
    "Migrants_2011", "Internal migrants_2011", "External migrants_2011", "Non-visible minority_2011",
    "Visible minority_2011", "Non-Canadian citizens_2011", "Canadian citizens_2011",
    "Non-immigrants_2011", "Immigrants_2011", "Non-permanent residents_2011", "Never married_2011",
    "Divorced_2011", "Widowed_2011", "Common-law couples_2011", "Married couples_2011",
    "Employed_2011", "Unemployed_2011",
    "No certificate or diploma_2011", "High school or Secondary degree_2011",
    "College or CEGEP degree_2011", "Trades certificate, diploma or apprenticeship_2011",
    "Bachelors degree or higher_2011",
    "Public transit_2011", "Walked_2011", "Bicycle_2011", "Transportation other methods_2011",
    "Transportation vehicle driver_2011", "Transportation vehicle non-driver_2011",
    "English only_2011", "French only_2011", "English and French_2011",
    "English and non-official language(s)_2011", "French and non-official language(s)_2011",
    "English, French, and non-official language(s)_2011", "Allophone_2011"
    ]

fields_dwe = [
    "Households_2011","Dwellings_2011", "Apartment with fewer than five stories_2011",
    "Apartment with five or more storeys_2011", "Row house_2011", "Single-detached house_2011",
    "Semi-detached house_2011", "Movable dwelling_2011", "Other single-attached house_2011",
    "Major repairs_2011","Regular maintenance_2011",
    "Owned housing_2011", "Rented housing_2011", "Band housing_2011",
    "Owner_2011", "Tenant_2011"
    ]

fields_weighted_avg = [
    "Median household income ($)_2011",
    "Average gross rent ($)_2011",
    "Average value dwelling ($)_2011"
    ]

fields_manual = [
    "Population change (%)_2011"
    ]


In [ ]:
# Combine all fields to convert to numeric
all_fields = fields_pop + fields_dwe + fields_weighted_avg + fields_manual

# Convert all columns except GeoUID to numeric
for col in all_fields:
  if col != ctuidcol:
    census_2011[col] = pd.to_numeric(census_2011[col], errors="coerce")

In [ ]:
# Remove placeholder CTUIDs (-1.0) before merging
cw_2011_2021 = cw_2011_2021[
    (cw_2011_2021['source_ctuid'] != -1.0) &
    (cw_2011_2021['target_ctuid'] != -1.0)
]

In [ ]:
# Joining the two input tables (following model code structure exactly)
merged = cw_2011_2021.merge(
    census_2011,
    how="outer",
    left_on="source_ctuid",
    right_on=ctuidcol,
    indicator=True
)

print(merged['_merge'].value_counts())

**w_pop and w_dwe**

In [ ]:
# Apply weights
output_fields = []

for f in fields_pop:
    weighted_col = f"w_{f}"
    merged[weighted_col] = merged["w_pop"] * merged[f]
    output_fields.append(weighted_col)

for f in fields_dwe:
    weighted_col = f"w_{f}"
    merged[weighted_col] = merged["w_dwe"] * merged[f]
    output_fields.append(weighted_col)

# Print numbers of rows and columns
print(merged.shape)

# Print column names
for col in merged.columns:
    print(col)

In [ ]:
# Group and aggregate weighted variables
weighted_data = merged.groupby("target_ctuid")[output_fields].agg("sum", min_count=1)

# Drop w_ prefix
weighted_data.columns = [c[2:] if c.startswith("w_") else c for c in weighted_data.columns]

# Print numbers of rows and columns
print(weighted_data.shape)

# Print column names
for col in weighted_data.columns:
    print(col)

**weighted average**

In [ ]:
# Initialize an empty DataFrame for weighted averages
weighted_avg_output = pd.DataFrame()

# For median household income (w_pop)
income_col = "Median household income ($)_2011"
weighted_income = (
    (merged[income_col] * merged["w_pop"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_pop"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[income_col] = weighted_income

# For average gross rent (w_dwe)
rent_col = "Average gross rent ($)_2011"
weighted_rent = (
    (merged[rent_col] * merged["w_dwe"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_dwe"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[rent_col] = weighted_rent

# For average dwelling value (w_dwe)
value_col = "Average value dwelling ($)_2011"
weighted_value = (
    (merged[value_col] * merged["w_dwe"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_dwe"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[value_col] = weighted_value

# Print column names
for col in weighted_avg_output.columns:
    print(col)

In [ ]:
# Combine weighted sums and weighed averages
final_2011_to_2021 = (
    weighted_data
    .join(weighted_avg_output, how="outer")
    .reset_index()
    .rename(columns={"target_ctuid": "GeoUID_2021"})
)

print(final_2011_to_2021.shape)
print(final_2011_to_2021.head())

**population change calculation**

In [ ]:
# Read in the standardized 2006 data
df2006 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/4_standardized_data/2006_standardized.csv')

# Merge 2001 population into the 2006-standardized table
final_2011_to_2021 = final_2011_to_2021.merge(
    df2006[['GeoUID_2021', 'Population_2006']],
    on="GeoUID_2021",
    how="left"
)

# Calculate % change (2006 → 2011)
den = pd.to_numeric(final_2011_to_2021["Population_2006"], errors="coerce").replace(0, np.nan)
num = pd.to_numeric(final_2011_to_2021["Population_2011"], errors="coerce")
final_2011_to_2021["Population change (%)_2011"] = ((num - den) / den * 100).round(1)

# Quick check
print(final_2011_to_2021[["GeoUID_2021", "Population_2006", "Population_2011", "Population change (%)_2011"]].head())

In [ ]:
final = final_2011_to_2021.copy()

# Drop useless 2006 columns
final = final.drop(columns=["Population_2006"], errors="ignore")

print("Finalized table:", final.shape)

for col in final.columns:
    print(col)

In [ ]:
# Export file
final.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/4_standardized_data/2011_standardized.csv', index=False)

# 2016 Standarization

In [ ]:
# Read in the data
census_2016 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2016_clean.csv')

# Read in crosswalk table
cw_2016_2021 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/3_census_crosswalks/ct_2016_2021.csv')

In [ ]:
# Define CTUID column name
ctuidcol = "GeoUID_2016"

# Categorize variables on calculation type (66 variables total excluding GeoUID and Area (sq km))
fields_pop = [
    "Population_2016",
    "Population 0 to 9_2016", "Population 10 to 19_2016", "Population 20 to 29_2016",
    "Population 30 to 39_2016", "Population 40 to 49_2016", "Population 50 to 59_2016",
    "Population 60 plus_2016", "Total male population_2016", "Total female population_2016",
    "Non-movers_2016", "Movers_2016", "Non-migrants_2016",
    "Migrants_2016", "Internal migrants_2016", "External migrants_2016", "Non-visible minority_2016",
    "Visible minority_2016", "Non-Canadian citizens_2016", "Canadian citizens_2016",
    "Non-immigrants_2016", "Immigrants_2016", "Non-permanent residents_2016", "Never married_2016",
    "Divorced_2016", "Widowed_2016", "Common-law couples_2016", "Married couples_2016",
    "Employed_2016", "Unemployed_2016",
    "No certificate or diploma_2016", "High school or Secondary degree_2016",
    "College or CEGEP degree_2016", "Trades certificate, diploma or apprenticeship_2016",
    "Bachelors degree or higher_2016",
    "Public transit_2016", "Walked_2016", "Bicycle_2016", "Transportation other methods_2016",
    "Transportation vehicle driver_2016", "Transportation vehicle non-driver_2016",
    "English only_2016", "French only_2016", "English and French_2016",
    "English and non-official language(s)_2016", "French and non-official language(s)_2016",
    "English, French, and non-official language(s)_2016", "Allophone_2016"
    ]

fields_dwe = [
    "Households_2016","Dwellings_2016", "Apartment with fewer than five stories_2016",
    "Apartment with five or more storeys_2016", "Row house_2016", "Single-detached house_2016",
    "Semi-detached house_2016", "Movable dwelling_2016", "Other single-attached house_2016",
    "Major repairs_2016","Regular maintenance_2016",
    "Owned housing_2016", "Rented housing_2016", "Band housing_2016",
    "Owner_2016", "Tenant_2016"
    ]

fields_weighted_avg = [
    "Median household income ($)_2016",
    "Average gross rent ($)_2016",
    "Average value dwelling ($)_2016"
    ]

fields_manual = [
    "Population change (%)_2016"
    ]


In [ ]:
# Combine all fields to convert to numeric
all_fields = fields_pop + fields_dwe + fields_weighted_avg + fields_manual

# Convert all columns except GeoUID to numeric
for col in all_fields:
  if col != ctuidcol:
    census_2016[col] = pd.to_numeric(census_2016[col], errors="coerce")

In [ ]:
# Remove placeholder CTUIDs (-1.0) before merging
cw_2016_2021 = cw_2016_2021[
    (cw_2016_2021['source_ctuid'] != -1.0) &
    (cw_2016_2021['target_ctuid'] != -1.0)
]

In [ ]:
# Joining the two input tables (following model code structure exactly)
merged = cw_2016_2021.merge(
    census_2016,
    how="outer",
    left_on="source_ctuid",
    right_on=ctuidcol,
    indicator=True
)

print(merged['_merge'].value_counts())

**w_pop and w_dwe**

In [ ]:
# Apply weights
output_fields = []

for f in fields_pop:
    weighted_col = f"w_{f}"
    merged[weighted_col] = merged["w_pop"] * merged[f]
    output_fields.append(weighted_col)

for f in fields_dwe:
    weighted_col = f"w_{f}"
    merged[weighted_col] = merged["w_dwe"] * merged[f]
    output_fields.append(weighted_col)

# Print numbers of rows and columns
print(merged.shape)

# Print column names
for col in merged.columns:
    print(col)

In [ ]:
# Group and aggregate weighted variables
weighted_data = merged.groupby("target_ctuid")[output_fields].agg("sum", min_count=1)

# Drop w_ prefix
weighted_data.columns = [c[2:] if c.startswith("w_") else c for c in weighted_data.columns]

# Print numbers of rows and columns
print(weighted_data.shape)

# Print column names
for col in weighted_data.columns:
    print(col)

**weighted average**

In [ ]:
# Initialize an empty DataFrame for weighted averages
weighted_avg_output = pd.DataFrame()

# For median household income (w_pop)
income_col = "Median household income ($)_2016"
weighted_income = (
    (merged[income_col] * merged["w_pop"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_pop"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[income_col] = weighted_income

# For average gross rent (w_dwe)
rent_col = "Average gross rent ($)_2016"
weighted_rent = (
    (merged[rent_col] * merged["w_dwe"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_dwe"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[rent_col] = weighted_rent

# For average dwelling value (w_dwe)
value_col = "Average value dwelling ($)_2016"
weighted_value = (
    (merged[value_col] * merged["w_dwe"])
    .groupby(merged["target_ctuid"])
    .agg("sum", min_count=1)
    / merged["w_dwe"].groupby(merged["target_ctuid"]).agg("sum", min_count=1)
)
weighted_avg_output[value_col] = weighted_value

# Print column names
for col in weighted_avg_output.columns:
    print(col)

In [ ]:
# Combine weighted sums and weighed averages
final_2016_to_2021 = (
    weighted_data
    .join(weighted_avg_output, how="outer")
    .reset_index()
    .rename(columns={"target_ctuid": "GeoUID_2021"})
)

print(final_2016_to_2021.shape)
print(final_2016_to_2021.head())

**population change calculation**

In [ ]:
# Read in the standardized 2011 data
df2011 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/4_standardized_data/2011_standardized.csv')

# Merge 2011 population into the 2016-standardized table
final_2016_to_2021 = final_2016_to_2021.merge(
    df2011[['GeoUID_2021', 'Population_2011']],
    on="GeoUID_2021",
    how="left"
)

# Calculate % change (2011 → 2016)
den = pd.to_numeric(final_2016_to_2021["Population_2011"], errors="coerce").replace(0, np.nan)
num = pd.to_numeric(final_2016_to_2021["Population_2016"], errors="coerce")
final_2016_to_2021["Population change (%)_2016"] = ((num - den) / den * 100).round(1)

# Quick check
print(final_2016_to_2021[["GeoUID_2021", "Population_2011", "Population_2016", "Population change (%)_2016"]].head())

In [ ]:
final = final_2016_to_2021.copy()

# Drop useless 2011 columns
final = final.drop(columns=["Population_2011","Population_2011_x","Population_2011_y","Population change (%)_2011"],errors="ignore")

print("Finalized table:", final.shape)

for col in final.columns:
    print(col)

In [ ]:
# Export file
final.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/4_standardized_data/2016_standardized.csv', index=False)